# 🦙 Fine-tune LLaMA 3.1 with Unsloth

This notebook demonstrates how to fine-tune LLaMA 3.1 8B for structured JSON output generation using QLoRA and Unsloth.

## Prerequisites
- Google Colab with GPU runtime (T4 or better)
- HuggingFace account with access to LLaMA 3.1

In [ ]:
# Install dependencies
!pip install -q unsloth
!pip install -q transformers datasets peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.7/405.7 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 806.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 136.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Imports
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## 1. Load Base Model with 4-bit Quantization

In [ ]:
# Configuration
max_seq_length = 2048
dtype = None  # Auto-detect
load_in_4bit = True

# Load model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print(f"Model loaded successfully!")

==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Model loaded successfully!


## 2. Add LoRA Adapters

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

print(f"LoRA adapters added!")
model.print_trainable_parameters()

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2026.1.4 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


LoRA adapters added!
trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5196


## 3. Load and Prepare Dataset

In [ ]:
# Define prompt template
PROMPT_TEMPLATE = """### Instruction:
You are an AI that converts natural language instructions into structured JSON action plans.
Given the following instruction, output a valid JSON with these fields:
- object: the object to manipulate
- initial_position: where the object currently is
- action: what to do (move, rotate, scale)
- target_position: the destination or target state

### Input:
{instruction}

### Response:
{output}"""

def format_prompts(examples):
    texts = []
    for instruction, output in zip(examples['instruction'], examples['output']):
        text = PROMPT_TEMPLATE.format(
            instruction=instruction,
            output=output
        )
        texts.append(text)
    return {"text": texts}

In [ ]:
# Load dataset (from local file or HuggingFace)
# Option 1: Load from local JSONL
dataset = load_dataset('json', data_files='/content/data/train.jsonl')

# Option 2: Create sample dataset for demo
# sample_data = {
#     "instruction": [
#         "Move the red box to the blue platform",
#         "Rotate the green sphere 90 degrees",
#         "Scale the yellow cube to twice its size",
#     ],
#     "output": [
#         '{"object": "red box", "initial_position": "floor", "action": "move", "target_position": "top of blue platform"}',
#         '{"object": "green sphere", "initial_position": "center", "action": "rotate", "target_position": "90 degrees clockwise"}',
#         '{"object": "yellow cube", "initial_position": "origin", "action": "scale", "target_position": "2x original size"}',
#     ]
# }

# from datasets import Dataset
# dataset = Dataset.from_dict(dataset)
dataset = dataset.map(format_prompts, batched=True)

print(f"Dataset size: {len(dataset['train'])}")
print(f"Sample:\n{dataset['train'][0]['text'][:500]}...")

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Dataset size: 800
Sample:
### Instruction:
You are an AI that converts natural language instructions into structured JSON action plans.
Given the following instruction, output a valid JSON with these fields:
- object: the object to manipulate
- initial_position: where the object currently is
- action: what to do (move, rotate, scale)
- target_position: the destination or target state

### Input:
Place orange pyramid on top shelf

### Response:
{"object": "orange pyramid", "initial_position": "bottom shelf", "action": "mo...


## 4. Training Configuration

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./outputs",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    max_steps=100,  # Increase for full training
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=10,
    save_steps=50,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=42,
)

In [ ]:
# Initialize trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=training_args,
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/800 [00:00<?, ? examples/s]

## 5. Train the Model

In [ ]:
# Start training
trainer_stats = trainer.train()

print(f"Training complete!")
print(f"Training time: {trainer_stats.metrics['train_runtime']:.2f} seconds")

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 800 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"
wandb: Using W&B in offline mode.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
10,1.386400
20,0.180500
30,0.119200
40,0.110300
50,0.105500
60,0.101900
70,0.098900
80,0.099800
90,0.097800
100,0.099000


wandb: WARNING URL not available in offline run
wandb: WARNING URL not available in offline run


train/epoch,▁▂▃▃▄▅▆▆▇██
train/global_step,▁▂▃▃▄▅▆▆▇██
train/grad_norm,█▂▁▁▁▁▁▁▁▁
train/learning_rate,██▇▆▅▅▄▃▂▁
train/loss,█▁▁▁▁▁▁▁▁▁
total_flos,4346724114874368.0
train/epoch,1
train/global_step,100
train/grad_norm,0.15364
train/learning_rate,0.0
train/loss,0.099


Training complete!
Training time: 404.69 seconds


## 6. Save the Model

In [ ]:
# Save LoRA adapters
model.save_pretrained("text-to-action-lora")
tokenizer.save_pretrained("text-to-action-lora")

print("Model saved to 'text-to-action-lora'")

Model saved to 'text-to-action-lora'


In [ ]:
# Optional: Save merged model for easier inference
# model.save_pretrained_merged("text-to-action-merged", tokenizer, save_method="merged_16bit")

config.json:   0%|          | 0.00/956 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/4 [00:00<?, ?it/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [06:06<00:00, 91.73s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [06:51<00:00, 102.88s/it]


Unsloth: Merge process complete. Saved to `/content/text-to-action-merged`


## 7. Quick Inference Test

In [ ]:
# Test inference
FastLanguageModel.for_inference(model)

test_instruction = "Move the purple cylinder to the corner"

test_prompt = f"""### Instruction:
You are an AI that converts natural language instructions into structured JSON action plans.
Given the following instruction, output a valid JSON with these fields:
- object: the object to manipulate
- initial_position: where the object currently is
- action: what to do (move, rotate, scale)
- target_position: the destination or target state

### Input:
{test_instruction}

### Response:
"""

inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    temperature=0.1,
    do_sample=True,
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response.split("### Response:")[-1].strip())

{"object": "purple cylinder", "initial_position": "desk", "action": "move", "target_position": "corner"}


In [ ]:
# model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [07:36<00:00, 114.15s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [06:45<00:00, 101.40s/it]


Unsloth: Merge process complete. Saved to `/content/gguf_model`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['meta-llama-3.1-8b-instruct.F16.gguf']
Un

{'save_directory': 'gguf_model',
 'gguf_files': ['meta-llama-3.1-8b-instruct.Q4_K_M.gguf'],
 'modelfile_location': '/content/Modelfile',
 'want_full_precision': False,
 'is_vlm': False,
 'fix_bos_token': False}

In [ ]:
# from google.colab import files
# import os

# gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
# if gguf_files:
#     gguf_file = os.path.join("gguf_model", gguf_files[0])
#     print(f"Downloading: {gguf_file}")
#     files.download(gguf_file)

## Next Steps

1. **Expand dataset** - Add more diverse instruction-action pairs
2. **Hyperparameter tuning** - Experiment with LoRA rank, learning rate
3. **Evaluation** - Run on held-out test set
4. **Deploy** - Export for Ollama or serve with FastAPI

In [ ]:
# Install backend dependencies
!pip install -q fastapi uvicorn pyngrok nest_asyncio
!pip install -q pydantic python-multipart

# Remove existing repo and re-clone for a clean state
!rm -rf text-to-action-llm
!git clone https://github.com/Rockstatata/text-to-action-llm.git
%cd text-to-action-llm/backend

Cloning into 'text-to-action-llm'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 60 (delta 7), reused 58 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (60/60), 71.76 KiB | 5.52 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/text-to-action-llm/backend


In [ ]:
# Create necessary directories if they don't exist for model.py
!mkdir -p app/llm

In [ ]:
# Ensure we are in the base content directory
%cd /content

# Change directory to the backend folder for API startup
%cd text-to-action-llm/backend

# Restart the model loading and API server

# Close any existing ngrok tunnels
# (ngrok will be re-imported below)
# ngrok.kill() # Commented out as ngrok is not yet imported

# Attempt to kill any existing process on port 8000 more robustly
import subprocess
try:
    # Find PIDs of processes listening on port 8000
    pids_output = subprocess.check_output(['lsof', '-ti', ':8000']).decode().strip()
    if pids_output:
        pids = pids_output.split('\n')
        print(f"Found processes on port 8000: {pids}. Attempting to kill them.")
        for pid in pids:
            subprocess.run(['kill', '-9', pid])
        import time
        time.sleep(3) # Give processes time to die
except (subprocess.CalledProcessError, FileNotFoundError):
    print("No processes found on port 8000 or lsof not available. Proceeding...")

# Clear Python kernel state to force module reloads
%reset -f

# Import ngrok AFTER kernel reset
from pyngrok import ngrok

# Now that ngrok is imported, we can kill any lingering tunnels
ngrok.kill()

# Set your ngrok authtoken here. Replace 'YOUR_NGROK_AUTHTOKEN' with your actual token.
# You can get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("37BlbkhM8x2qDypKgoFjwKkcvmm_3Eba4AfrSmLFhRHqzPevf") # <-- ADD THIS LINE

# Re-run Juyv6zb7Qd-C to load the model with updated logic
import os
os.environ["LLM_BACKEND"] = "transformers"
os.environ["MODEL_PATH"] = "/content/text-to-action-lora"  # Adjust path if needed

from app.llm.model import load_model
load_model()

import nest_asyncio
import uvicorn
from threading import Thread

nest_asyncio.apply()

from app.main import app

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = Thread(target=run, daemon=True)
thread.start()

import time
time.sleep(5) # Give it more time to start up

public_url = ngrok.connect(8000)
print(f"🚀 API live at: {public_url}")


/content
/content/text-to-action-llm/backend
No processes found on port 8000 or lsof not available. Proceeding...
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2026.1.4 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.
INFO:     Started server process [30592]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🚀 API live at: NgrokTunnel: "https://anthroponomical-jodi-mercilessly.ngrok-free.dev" -> "http://localhost:8000"
INFO:     35.230.72.166:0 - "GET /health HTTP/1.1" 200 OK
{'status': 'ok', 'model_loaded': True}
2026-01-26 22:07:16 | INFO     | text-to-action | Received instruction: Move red box to platform


INFO:text-to-action:Received instruction: Move red box to platform


INFO:     35.230.72.166:0 - "POST /api/infer HTTP/1.1" 200 OK

--- Inference Test Result ---
{'object': 'red box', 'initial_position': 'current location', 'action': 'move', 'target_position': 'platform'}


In [ ]:
import requests
print(f"🚀 API live at: {public_url}")

# Perform inference test here

response1 = requests.get(f"{public_url.public_url}/health")
print(response1.json())
response = requests.post(f"{public_url.public_url}/api/infer", json={"instruction": "Move red box to platform"})

if response.status_code == 200:
    print("\n--- Inference Test Result ---")
    print(response.json())
else:
    print("\n--- Inference Test Failed ---")
    print(f"Error: {response.status_code}")
    print(response.text)

🚀 API live at: NgrokTunnel: "https://anthroponomical-jodi-mercilessly.ngrok-free.dev" -> "http://localhost:8000"
INFO:     35.230.72.166:0 - "GET /health HTTP/1.1" 200 OK
{'status': 'ok', 'model_loaded': True}
2026-01-26 22:07:40 | INFO     | text-to-action | Received instruction: Move red box to platform


INFO:text-to-action:Received instruction: Move red box to platform


INFO:     35.230.72.166:0 - "POST /api/infer HTTP/1.1" 200 OK

--- Inference Test Result ---
{'object': 'red box', 'initial_position': 'current location', 'action': 'move', 'target_position': 'platform'}
